In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
# Text file data converted to integer data type
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Colab\ Notebooks/AI_project


Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
#Load txt file
def getRawData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
#turns [0,0,0,1] into 4 for directions zero default 
def getDir(line):
	if(line[0] == '1'):
		return 1
	if(line[1] == '1'):
		return 2
	if(line[3] == '1'):
		return 3
	if(line[4] == '1'):
		return 4
	return 0 

In [ ]:
def genMatricies(lines):
	x_data = []
	x_row_list = []
	y_data = []

	for x in range(0,len(lines)):
		# This is used to get rid of the \n in the txt file
		if (x % 12 == 0):
			first = 0
			continue
		#/the 51th line is the direction vector
		if (x % 12 == 11):
			y_data.append(getDir(lines[x]))
			
		#everything else is the matrix
		else:
			#get each char we then convert it to string for tensorflow calcs
			for sym in lines[x]:
				i = 0
				if first == 0:
					i = 1
					first = 1

				if(sym == 'X'):
					i = -1
				elif(sym == '\n'):
					continue
				elif(sym == '2'):
					i = 2
				elif(sym == '1'):
					i = 1
				x_row_list.append(i)
			#when we have 2500 chars we know we have a 50x50 make it an array and add it to list
			if(len(x_row_list) == 100):
				x_data.append(np.array(x_row_list).reshape(10,10))
				x_row_list = []
			# turn list into numpy array, turn numpy array into catagorical input
	return (np.array(x_data),tf.keras.utils.to_categorical(np.array(y_data), 4))

DATA CREATION

In [ ]:
lines = getRawData("proto10_train.txt")
x1 ,y1 = genMatricies(lines)

In [ ]:
np.save('proto10_train_x1.npy', x1)


In [ ]:
np.save('proto10_train_y1.npy', y1)

In [ ]:
x1 = np.load("proto10_train_x1.npy")
y1 = np.load("proto10_train_y1.npy")

In [ ]:
train_x1 = np.load('proto50_500_train_x1.npy') 
#train_y1 = np.load('proto50_train_y1.npy')

In [ ]:
train_y1 = np.load('proto50_500_train_y1.npy')
#train_y2 = np.load('proto50_train_y2.npy')
#total_y = np.append(train_y1, train_y2)
#np.save('proto50_train_total_y.npy', total_y)

TRAINING AGENT

In [ ]:
print(tf.config.list_physical_devices('GPU'))
#checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto50_model/proto50_train"
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/"
checkpoint_dir = os.path.dirname(checkpoint_path)
EPOCHS_DEFAULT = 3

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:

def buildModel():
  global EPOCHS_DEFAULT
  # add more layers here as needed
  gridInput = tf.keras.layers.Input(shape = (10,10))
  flatten_image = tf.keras.layers.Flatten()( gridInput )

  #dense_1 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( flatten_image )
  #dense_2 = tf.keras.layers.Dense( units = 400, activation = tf.nn.relu )( dense_1 )
  #dense_3 = tf.keras.layers.Dense( units = 200, activation = tf.nn.relu )( dense_2 )
  #dense_4 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( dense_3 )

  #infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )

  dense_1 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( flatten_image )

  infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_1 )

  probabilities = tf.keras.layers.Softmax()( infDec )

  model = tf.keras.Model( inputs = gridInput, outputs = probabilities)
  model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
  latest = tf.train.latest_checkpoint(checkpoint_dir)
 
  if(latest == None):
    return model

  model.load_weights(latest)
  EPOCHS_DEFAULT = 3

  return model

In [ ]:
# this allows you to resume or expand upon training
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
tf.test.is_gpu_available()

True

In [ ]:
model = buildModel()

In [ ]:
#model = buildModel()

#model = tf.keras.models.load_model(checkpoint_path)

history = model.fit(x1 , y1, epochs = EPOCHS_DEFAULT,
          callbacks=[cp_callback] )


Epoch 1/3
26976/26983 [============================>.] - ETA: 0s - loss: 0.6169 - accuracy: 0.7311
Epoch 00001: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 62s 2ms/step - loss: 0.6168 - accuracy: 0.7311
Epoch 2/3
26965/26983 [============================>.] - ETA: 0s - loss: 0.4541 - accuracy: 0.8007
Epoch 00002: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 62s 2ms/step - loss: 0.4540 - accuracy: 0.8007
Epoch 3/3
26975/26983 [============================>.] - ETA: 0s - loss: 0.4189 - accuracy: 0.8122
Epoch 00003: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 62s 2ms/step - loss: 0.4189 - accuracy: 0.8122


TESTING

In [ ]:
def getTestData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
test_lines = getTestData("proto10_test.txt")

tx,ty = genMatricies(test_lines)

KeyboardInterrupt: ignored

In [ ]:
np.save('proto10_test_x1.npy', tx)
np.save('proto10_test_y1.npy', ty)

In [ ]:
tx = np.load('proto10_test_x1.npy')
ty = np.load('proto10_test_y1.npy')

In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(x1, y1, validation_data=(tx, ty), epochs=3)

Epoch 1/3
26983/26983 [==============================] - 83s 3ms/step - loss: 0.4014 - accuracy: 0.8171 - val_loss: 0.6506 - val_accuracy: 0.7217
Epoch 2/3
26983/26983 [==============================] - 72s 3ms/step - loss: 0.3893 - accuracy: 0.8222 - val_loss: 0.6814 - val_accuracy: 0.7179
Epoch 3/3
26983/26983 [==============================] - 82s 3ms/step - loss: 0.3816 - accuracy: 0.8241 - val_loss: 0.7004 - val_accuracy: 0.7294


GEN MAZE

In [ ]:
#map constructor
from random import randint
from matplotlib import pyplot as plot


In [ ]:
#size of each grid
grid_size = 10
#dim x dim for maze
size = int(10)
#prob of wall appearance (0~1)
p_wall = 30

In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]

In [ ]:
def createMaze():

  x_data = []
  x_row_list = []

  for row in range(size):
      for col in range(size):
          s = randint(1,100)
          p = p_wall
          if s<=p:
            maze[row][col] = 'X'
          else:
            maze[row][col] = '0'
            
  maze[0][0]= '0'
  maze[size-1][size-1] = '2'

In [ ]:
createMaze()

In [ ]:
print(maze)

[['0', 'X', '0', 'X', 'X', '0', 'X', '0', '0', 'X'], ['0', 'X', '0', '0', 'X', '0', '0', '0', '0', '0'], ['X', '0', '0', '0', 'X', '0', '0', 'X', 'X', 'X'], ['X', '0', '0', '0', '0', '0', 'X', 'X', '0', 'X'], ['0', '0', 'X', '0', '0', '0', 'X', '0', '0', 'X'], ['0', '0', '0', '0', '0', 'X', '0', '0', 'X', 'X'], ['0', '0', 'X', 'X', '0', '0', 'X', '0', '0', 'X'], ['X', 'X', '0', '0', '0', '0', '0', 'X', '0', '0'], ['0', '0', '0', 'X', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', 'X', '0', '0', 'X', '0', '0', '2']]


In [ ]:
def genMaze(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return np.array(x_data)

In [ ]:
def getDir(y_arr):
  max = -9
  idx = 0
  curr = 0
  for i in range(len(y_arr)):
    if y_arr[i] > max:
      idx = i
  
  dir = [0,0,0,0]
  dir[idx] = 1
  
  
  return dir

In [ ]:
new_maze = genMaze(size)

In [ ]:
print(new_maze)

[[[ 0  0  0  0  0  0  0 -1  0 -1]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0 -1 -1]
  [-1  0 -1  0  0 -1  0  0  0 -1]
  [ 0 -1 -1  0 -1  0 -1 -1 -1  0]
  [ 0  0  0  0  0  0 -1  0 -1  0]
  [-1  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]]


In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]
createMaze()
new_maze = genMaze(size)
new_maze[0][0][0] = 1
print(new_maze)

[[[ 1  0  0  0  0  0  0 -1  0 -1]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0 -1 -1]
  [-1  0 -1  0  0 -1  0  0  0 -1]
  [ 0 -1 -1  0 -1  0 -1 -1 -1  0]
  [ 0  0  0  0  0  0 -1  0 -1  0]
  [-1  0  0  0  0  0  0  0  0  0]
  [-1  0  0 -1  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]]


In [ ]:
print(tx[:15])
direct = model.predict(tx[:15])

[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 ...

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1 -1  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]]


In [ ]:
print(direct[:15])

[[8.5528847e-03 4.8189278e-05 7.7087522e-01 2.2052372e-01]
 [5.3488437e-13 9.3615345e-05 8.1631157e-04 9.9909008e-01]
 [3.7918011e-07 3.6593428e-04 2.4853030e-01 7.5110340e-01]
 [3.4743791e-28 5.2565726e-11 4.1866964e-29 1.0000000e+00]
 [1.2598440e-14 5.8334724e-08 1.3909444e-07 9.9999988e-01]
 [2.8214382e-04 1.8242616e-04 1.0351812e-01 8.9601737e-01]
 [1.0801328e-09 6.2273791e-08 6.4831853e-01 3.5168138e-01]
 [1.4048910e-09 6.0585211e-05 1.3777557e-02 9.8616195e-01]
 [7.2407082e-04 8.5876498e-05 1.3980719e-01 8.5938287e-01]
 [1.9398764e-12 5.6124358e-03 3.0397812e-05 9.9435717e-01]
 [6.4113208e-11 1.2884502e-10 9.9372226e-01 6.2777568e-03]
 [1.8183006e-06 9.3414092e-06 4.4354968e-02 9.5563394e-01]
 [2.3116110e-14 2.9615776e-15 4.0550575e-01 5.9449428e-01]
 [2.2757077e-04 1.4553044e-06 1.6875135e-02 9.8289585e-01]
 [1.5442468e-04 1.1517259e-09 1.6363930e-03 9.9820912e-01]]


In [ ]:
for i in range(15):
  print(getDir(direct[i]))

[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]


In [ ]:
print(tx[:10])

[[[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  